In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

# os.environ["OPENAI_API_TYPE"] = "azure"
# os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
# # os.environ["OPENAI_API_BASE"] = "https://test0406.openai.azure.com/openai/deployments/test0406/"
# os.environ["OPENAI_API_KEY"] = "c9dd1a32c336407caef07fe731248625"

True

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import GPT4All
from langchain import HuggingFaceHub, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import AzureOpenAI

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap  = 30,
    length_function = len,
)

# initialize Hub LLM
# llm = HuggingFaceHub(repo_id='THUDM/chatglm-6b', model_kwargs={"trust_remote_code": True})
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10})
# llm = ChatGLMService()
# llm.load_model()

embeddings = OpenAIEmbeddings()
# embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                            model_kwargs={"device": "cuda"})
persist_directory = 'rpmv2'
txt_docsearch = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=3500)
# llm = GPT4All(model="/home/jinzy/Downloads/ggml-gpt4all-j-v1.3-groovy.bin", n_ctx=512, n_threads=8)
# llm = AzureOpenAI(
#     deployment_name="test0406",
#     model_name="gpt-3.5", 
# )

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


rewritte_template = """根据聊天记录和问题，把问题重写为一个独立的、完整的问题。
聊天记录:
{chat_history}
问题: {question}
独立问题:"""

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_template(rewritte_template)
# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt)

refine_template = """请检查下述检索结果是否可以用于更新回复内容。
检索结果:
{context_str}

如果这个检索结果包含了有用的信息，请根据它和原始回复内容重新生成更好的回复。如果没有帮助，就返回原始回复内容。请记住，总是生成中文回复内容。
更好的回复："""
# (
#     "We have the opportunity to refine the existing answer"
#     "(only if needed) with some more context below.\n"
#     "------------\n"
#     "{context_str}\n"
#     "------------\n"
#     "Given the new context, refine the original answer to better "
#     "answer the question. "
#     "If the context isn't useful, return the original answer."
# )
refine_prompt = ChatPromptTemplate.from_template(refine_template)
doc_chain = load_qa_chain(llm, chain_type="refine", refine_prompt =refine_prompt)

summary_template = """根据聊天内容增量式生成摘要，在原始摘要上附加新的摘要。
EXAMPLE
当前摘要: 
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

新的聊天内容:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

新的摘要:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

当前摘要:
{summary}

新的聊天内容:
{new_lines}

新的摘要:
"""

summary_prompt = PromptTemplate(input_variables=["summary", "new_lines"], template=summary_template)

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationSummaryBufferMemory(llm=llm, prompt = summary_prompt,
        max_token_limit=1000, memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain(
    retriever=txt_docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory
)
# chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=txt_docsearch.as_retriever(), 
#                                               chain_type='refine', memory=memory)

In [22]:
query = "请用简单的语言尽可能详细地解释一下RPM产品中的任务挖掘功能"
result = chain({"question": query})
print(result['answer'])

InvalidRequestError: Resource not found

In [5]:
query = "请给出一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
#print(result)
print(result['answer'])

提出的RPM流程旨在发现可以由RPA机器人以端到端方式执行的例程。然而，这种假设是有限制的。实际上，例程可能只能针对特定的情况进行自动化，而不是所有情况都能自动化（即，自动化可能只能部分实现）。除了提出的RPM流程之外，一个关键的挑战是如何发现部分确定性例程。因此，在实施自动化过程中，需要考虑到这些限制和挑战。


In [6]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})

print(result['answer'])

RPM工具的数据主要包括用户交互(UI)日志。RPM旨在帮助分析师绘制候选自动化例程的系统清单，并帮助他们生成可执行的规范，作为自动化的起点。因此，RPM专注于RPA生命周期的前两个阶段：分析和开发。


In [7]:
result

{'question': '请介绍一个RPM产品的实际案例',
 'chat_history': [HumanMessage(content='请用简单的语言尽可能详细地解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False),
  AIMessage(content='最近出现了一种新的技术家族，叫做机器人流程挖掘（Robotic Process Mining，RPM）。RPM旨在帮助RPA开发人员和分析师在RPA生命周期的早期阶段。具体而言，它可以帮助分析师系统地列出可以使用RPA自动化的任务清单，并综合出可执行的任务规范，作为起点。', additional_kwargs={}, example=False),
  HumanMessage(content='请给出一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='提出的RPM流程旨在发现可以由RPA机器人以端到端方式执行的例程。然而，这种假设是有限制的。实际上，例程可能只能针对特定的情况进行自动化，而不是所有情况都能自动化（即，自动化可能只能部分实现）。除了提出的RPM流程之外，一个关键的挑战是如何发现部分确定性例程。因此，在实施自动化过程中，需要考虑到这些限制和挑战。', additional_kwargs={}, example=False),
  HumanMessage(content='请介绍一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='RPM工具的数据主要包括用户交互(UI)日志。RPM旨在帮助分析师绘制候选自动化例程的系统清单，并帮助他们生成可执行的规范，作为自动化的起点。因此，RPM专注于RPA生命周期的前两个阶段：分析和开发。', additional_kwargs={}, example=False)],
 'answer': 'RPM工具的数据主要包括用户交互(UI)日志。RPM旨在帮助分析师绘制候选自动化例程的系统清单，并帮助他们生成可执行的规范，作为自动化的起点。因此，RPM专注于RPA生命周期的前两个阶段：分析和开发。'}

In [ ]:
# query = "请介绍一个RPM产品的实际案例"
# #result = chain({"question": query, "chat_history": chat_history})
# result = chain({"question": query + ' 请用中文回答。'})

# print(result['answer'])

In [ ]:
# result

In [ ]:
# chain.memory

In [ ]:
# from typing import List, Optional
# from langchain.llms.base import LLM
# from langchain.llms.utils import enforce_stop_tokens
# from transformers import AutoModel, AutoTokenizer


# class ChatGLMService(LLM):
#     max_token: int = 10000
#     temperature: float = 0.1
#     top_p = 0.9
#     history = []
#     tokenizer: object = None
#     model: object = None

#     def __init__(self):
#         super().__init__()

#     @property
#     def _llm_type(self) -> str:
#         return "ChatGLM"

#     def _call(self,
#               prompt: str,
#               stop: Optional[List[str]] = None) -> str:
#         response, _ = self.model.chat(
#             self.tokenizer,
#             prompt,
#             history=self.history,
#             max_length=self.max_token,
#             temperature=self.temperature,
#         )
#         if stop is not None:
#             response = enforce_stop_tokens(response, stop)
#         self.history = self.history + [[None, response]]
#         return response

#     def load_model(self, model_name_or_path: str = "THUDM/chatglm-6b"):
#         self.tokenizer = AutoTokenizer.from_pretrained(
#             model_name_or_path,
#             trust_remote_code=True
#         )
#         print('加载chatglm-6b', model_name_or_path)
#         # 这里使用的是 量化后的模型int4
#         self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).float().half().cuda()
#         #.half().quantize(4).cuda()
#         #.half().cuda()
#         self.model=self.model.eval()

In [ ]:
# import chromadb
# import chromadb.config

# persist_directory = 'rpmv2'
# txt_docsearch._client_settings = chromadb.config.Settings(
#                     chroma_db_impl="duckdb+parquet", persist_directory = persist_directory
#                 )
# txt_docsearch._client = chromadb.Client(txt_docsearch._client_settings)
# txt_docsearch._persist_directory = persist_directory
# txt_docsearch.persist()